In [1]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Введите ваш OpenAI API ключ: ")

Введите ваш OpenAI API ключ:  ········


In [22]:
import arxiv
import requests
from langchain.chains import RetrievalQA
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI as OpenAI
from pydantic import BaseModel, Field
from smolagents import CodeAgent, OpenAIServerModel, tool

# Заводим вспомогательные функции для инструментов

In [3]:
class ArxivArticle(BaseModel):
    title: str = Field(description="Заголовок статьи")
    summary: str = Field(description="Краткое содержание статьи")
    pdf_url: str = Field(description="Ссылка на PDF-файл статьи")


def search_arxiv(query: str) -> ArxivArticle:
    search = arxiv.Search(query=query, max_results=1)
    result = next(search.results())

    # Проверяем, есть ли pdf_url
    pdf_url = getattr(result, "pdf_url", None)
    if not pdf_url:
        # Формируем pdf_url вручную из entry_id, если не найден
        entry_id = result.entry_id  # например https://arxiv.org/abs/2301.12345
        # извлечь id из entry_id
        arxiv_id = entry_id.split("/")[-1]
        pdf_url = f"https://arxiv.org/pdf/{arxiv_id}.pdf"

    return ArxivArticle.model_validate(
        {"title": result.title, "summary": result.summary, "pdf_url": pdf_url}
    )

In [4]:
def download_pdf(pdf_url: str) -> str:
    """Скачивает pdf, сохраняет в temp.pdf."""
    r = requests.get(pdf_url)
    with open("temp.pdf", "wb") as f:
        f.write(r.content)
    return "temp.pdf"


def load_pdf(pdf_path: str):
    """Загружает PDF и разбивает на страницы."""
    loader = PyPDFLoader(pdf_path)
    return loader.load_and_split()

In [5]:
class PDFQA:
    """Класс для индексации PDF и ответа на вопросы по статье."""

    def __init__(self):
        self.qa_chain = None

    def index_pdf(self, pages):
        embeddings = OpenAIEmbeddings()
        vectorstore = FAISS.from_documents(pages, embeddings)
        retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
        self.qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), retriever=retriever)

    def ask(self, question: str) -> str:
        if not self.qa_chain:
            return "PDF не загружен и не проиндексирован."
        return self.qa_chain.run(question)


pdf_qa = PDFQA()

In [6]:
def summarize_pdf(pdf_path: str) -> str:
    pages = load_pdf(pdf_path)
    llm = OpenAI(model_name="o3-mini")
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    summary = chain.invoke(pages)
    return summary

# Определяем инструменты для агента

In [16]:
@tool
def tool_search_arxiv(query: str) -> str:
    """Tool for searching arxiv articles by text query.
    Args:
        query: the topic to search for
    """
    return search_arxiv(query).pdf_url


@tool
def tool_load_and_index(pdf_url: str) -> str:
    """Tool for downloading a PDF file of an article from a link.
    Args:
        pdf_url: Arxiv paper url
    """
    pdf_path = download_pdf(pdf_url)
    pages = load_pdf(pdf_path)
    pdf_qa.index_pdf(pages)
    return f"PDF загружен и проиндексирован: {len(pages)} страниц"


@tool
def tool_ask_pdf(question: str) -> str:
    """A tool for answering questions about an uploaded article.
    Args:
        question: Question about the article
    """
    return pdf_qa.ask(question)


@tool
def tool_summarize(pdf_url: str) -> str:
    """Tool for summarizing the uploaded article.
    Args:
        pdf_url: Arxiv paper url
    """
    pdf_path = download_pdf(pdf_url)
    return summarize_pdf(pdf_path)["output_text"]

# Сам агент

In [18]:
agent = CodeAgent(
    tools=[tool_summarize, tool_ask_pdf, tool_load_and_index, tool_search_arxiv],
    model=OpenAIServerModel(model_id="o3-mini"),
)

# Примеры

In [12]:
agent.run("diffusion models 2024")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ diffusion models 2024                                                                                           │
│                                                                                                                 │
╰─ OpenAIServerModel - o3-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  results = tool_search_arxiv(query="diffusion models 2024")                                                       
  final_answer(results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

/var/folders/t7/0zcbppxn02b4wfhmpbfg0f2m0000gn/T/ipykernel_24024/2403004161.py:8: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  result = next(search.results())


Out - Final answer: http://arxiv.org/pdf/2406.18241v1

[Step 1: Duration 12.27 seconds| Input tokens: 2,090 | Output tokens: 207]

'http://arxiv.org/pdf/2406.18241v1'

In [20]:
agent.run("find diffusion models 2024 paper and give its summary")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ find diffusion models 2024 paper and give its summary                                                           │
│                                                                                                                 │
╰─ OpenAIServerModel - o3-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Search for the diffusion models 2024 paper on arXiv.                                                   
  search_results = tool_search_arxiv(query="diffusion models 2024")                                                
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

/var/folders/t7/0zcbppxn02b4wfhmpbfg0f2m0000gn/T/ipykernel_24024/2403004161.py:8: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  result = next(search.results())


Execution logs:
http://arxiv.org/pdf/2406.18241v1

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Magnus Svärd’s note defends the diffusive compressible Euler (dcE) model against criticisms by M.  
  Morris. Svärd clarifies that the dcE model—designed for ideal gases—incorporates additional diffusive terms      
  (with tunable parameters like ν and κT) into the conservation laws to account for heat diffusion and sound       
  attenuation. He emphasizes that these parameters are adjusted to experimental data rather than being             
  fundamental constants, and he distinguishes modeled diffusion from viscous conduction. The note argues that      
  while the standard Navier-Stokes-Fourier (NSF) model can match experiments via its bulk viscosity parameter,     
  its own limitations (e.g., uncertainties in bulk viscosity and issues at high ultrasound frequencies) parallel   
  those found in the dcE approach. Svärd also addresses criticisms regarding frame dependence and applicability,   
  asserting that the dcE model remains robust, particularly for ideal gases, while acknowledging that neither      
  model perfectly captures strongly non-ideal or polyatomic gas behavior at extreme conditions. Overall, the note  
  reinforces the validity and ongoing development of the dcE model as a competitive alternative to the NSF system  
  for numerical simulations of compressible, diffusive ideal gases.")                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Magnus Svärd’s note defends the diffusive compressible Euler (dcE) model against criticisms by 
M. Morris. Svärd clarifies that the dcE model—designed for ideal gases—incorporates additional diffusive terms 
(with tunable parameters like ν and κT) into the conservation laws to account for heat diffusion and sound 
attenuation. He emphasizes that these parameters are adjusted to experimental data rather than being fundamental 
constants, and he distinguishes modeled diffusion from viscous conduction. The note argues that while the standard 
Navier-Stokes-Fourier (NSF) model can match experiments via its bulk viscosity parameter, its own limitations 
(e.g., uncertainties in bulk viscosity and issues at high ultrasound frequencies) parallel those found in the dcE 
approach. Svärd also addresses criticisms regarding frame dependence and applicability, asserting that the dcE 
model remains robust, particularly for ideal gases, while acknowledging that neither model perfectly captures 
strongly non-ideal or polyatomic gas behavior at extreme conditions. Overall, the note reinforces the validity and 
ongoing development of the dcE model as a competitive alternative to the NSF system for numerical simulations of 
compressible, diffusive ideal gases.

[Step 3: Duration 5.12 seconds| Input tokens: 7,174 | Output tokens: 1,750]

'Magnus Svärd’s note defends the diffusive compressible Euler (dcE) model against criticisms by M. Morris. Svärd clarifies that the dcE model—designed for ideal gases—incorporates additional diffusive terms (with tunable parameters like ν and κT) into the conservation laws to account for heat diffusion and sound attenuation. He emphasizes that these parameters are adjusted to experimental data rather than being fundamental constants, and he distinguishes modeled diffusion from viscous conduction. The note argues that while the standard Navier-Stokes-Fourier (NSF) model can match experiments via its bulk viscosity parameter, its own limitations (e.g., uncertainties in bulk viscosity and issues at high ultrasound frequencies) parallel those found in the dcE approach. Svärd also addresses criticisms regarding frame dependence and applicability, asserting that the dcE model remains robust, particularly for ideal gases, while acknowledging that neither model perfectly captures strongly non-

In [24]:
agent.run(
    "What is the research methodology used in the article Rebuttal of Morris’ criticism of the diffusive compressible Euler model"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the research methodology used in the article Rebuttal of Morris’ criticism of the diffusive             │
│ compressible Euler model                                                                                        │
│                                                                                                                 │
╰─ OpenAIServerModel - o3-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Search for the article on arXiv                                                                        
  search_result = tool_search_arxiv(query="Rebuttal of Morris’ criticism of the diffusive compressible Euler       
  model")                                                                                                          
  print("Search result:", search_result)                                                                           
                                                                                                                   
  # Assuming the search_result contains a PDF URL link for the article, we will load and index it next.            
  # Normally, one would extract the pdf_url from the search result; for now we assume the search result is the     
  pdf_url:                                                                                                         
  pdf_url = search_result  # assuming search_result returns the pdf_url                                            
                                                                                                                   
  # Step 2: Load and index the article using its PDF URL                                                           
  loading_output = tool_load_and_index(pdf_url=pdf_url)                                                            
  print("Loading and indexing output:", loading_output)                                                            
                                                                                                                   
  # Step 3: Ask a specific question about the research methodology used in the article.                            
  methodology_info = tool_ask_pdf(question="What is the research methodology used in the article 'Rebuttal of      
  Morris’ criticism of the diffusive compressible Euler model'?")                                                  
  print("Methodology info:", methodology_info)                                                                     
                                                                                                                   
  # For the final answer, I provide the extracted methodology info.                                                
  final_answer(methodology_info)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

/var/folders/t7/0zcbppxn02b4wfhmpbfg0f2m0000gn/T/ipykernel_24024/2403004161.py:8: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  result = next(search.results())
/var/folders/t7/0zcbppxn02b4wfhmpbfg0f2m0000gn/T/ipykernel_24024/4223951.py:15: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self.qa_chain.run(question)


Execution logs:
Search result: http://arxiv.org/pdf/2406.18241v1
Loading and indexing output: PDF загружен и проиндексирован: 8 страниц
Methodology info: The research methodology used in the article 'Rebuttal of Morris’ criticism of the diffusive 
compressible Euler model' is primarily theoretical in nature. The author, Magnus Svärd, addresses and rebuts 
Morris' criticism through theoretical arguments and mathematical formulations related to the diffusive compressible
Euler model, highlighting the specific equations, terms, and parameters involved in the model. The rebuttal 
includes discussions on the model's equations, assumptions, and parameters, as well as comparisons with other 
models like the Navier-Stokes-Fourier (NSF) system. The methodology involves mathematical analysis, theoretical 
reasoning, and critical examination of Morris' criticisms to clarify and defend the diffusive compressible Euler 
model.

Out - Final answer: The research methodology used in the article 'Rebuttal of Morris’ criticism of the diffusive 
compressible Euler model' is primarily theoretical in nature. The author, Magnus Svärd, addresses and rebuts 
Morris' criticism through theoretical arguments and mathematical formulations related to the diffusive compressible
Euler model, highlighting the specific equations, terms, and parameters involved in the model. The rebuttal 
includes discussions on the model's equations, assumptions, and parameters, as well as comparisons with other 
models like the Navier-Stokes-Fourier (NSF) system. The methodology involves mathematical analysis, theoretical 
reasoning, and critical examination of Morris' criticisms to clarify and defend the diffusive compressible Euler 
model.

[Step 1: Duration 43.24 seconds| Input tokens: 2,108 | Output tokens: 1,050]

"The research methodology used in the article 'Rebuttal of Morris’ criticism of the diffusive compressible Euler model' is primarily theoretical in nature. The author, Magnus Svärd, addresses and rebuts Morris' criticism through theoretical arguments and mathematical formulations related to the diffusive compressible Euler model, highlighting the specific equations, terms, and parameters involved in the model. The rebuttal includes discussions on the model's equations, assumptions, and parameters, as well as comparisons with other models like the Navier-Stokes-Fourier (NSF) system. The methodology involves mathematical analysis, theoretical reasoning, and critical examination of Morris' criticisms to clarify and defend the diffusive compressible Euler model."